# Concrete Image Classification with Transfer Learning
# Binary classification using MobileNetV2 + Fine-tuning

## 1. Introduction & Problem Statement

### Objective
Classify concrete surface images into two categories: Positive (defect/crack present) 
and Negative (no defect). This automated classification can assist in infrastructure 
inspection and maintenance workflows.

### Dataset Overview
- **Total Images:** 40,000 (20,000 per class)
- **Source:** 458 high-resolution images (4032×3024) augmented to create dataset
- **Image Size:** 227×227 pixels, RGB channels
- **Split:** 70% train, 15% validation, 15% test
- **Classes:** Negative (no defect), Positive (defect present)

## 2. Setup & Imports

In [9]:
import os, json, random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import pathlib
from pathlib import Path

In [10]:
# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')

## 3. Reproducibility Configuration

In [11]:
def set_seed(seed=13):
    """Ensure reproducible results across runs"""
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.keras.utils.set_random_seed(seed)
    tf.config.experimental.enable_op_determinism()

set_seed(13)
print("Reproducibility configured with seed=13")

Reproducibility configured with seed=13


## 4. Data Loading & Exploration

In [14]:
# Configure paths
DATA_ROOT = "data"
SPLITS_DIR = "splits"
OUTPUT_DIR = "results"

os.makedirs(SPLITS_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

def list_images(data_root):
    """List all images with their labels"""
    data_root = pathlib.Path(data_root)
    items = []
    for label_name in sorted(["Negative", "Positive"]):
        for ext in ("*.jpg", "*.png", "*.jpeg"):
            for p in (data_root / label_name).glob(ext):
                items.append((str(p), 0 if label_name=="Negative" else 1))
    return items

## 5. Visualize Sample Images

In [ ]:
def display_samples(data_root, n_samples=4):
    """Show sample images from each class"""
    fig, axes = plt.subplots(2, n_samples, figsize=(12, 6))
    
    for class_idx, class_name in enumerate(["Negative", "Positive"]):
        class_dir = pathlib.Path(data_root) / class_name
        sample_images = list(class_dir.glob("*.jpg"))[:n_samples]
        
        for i, img_path in enumerate(sample_images):
            img = plt.imread(img_path)
            axes[class_idx, i].imshow(img)
            axes[class_idx, i].axis('off')
            if i == 0:
                axes[class_idx, i].set_title(f"{class_name}", fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(f"{OUTPUT_DIR}/sample_images.png", dpi=120, bbox_inches='tight')
    plt.show()

# Display samples
display_samples(DATA_ROOT, n_samples=4)

In [ ]:
# Load and display dataset statistics
items = list_images(DATA_ROOT)
labels = [lbl for _, lbl in items]

print(f"Total images: {len(items)}")
print(f"Negative samples: {labels.count(0)}")
print(f"Positive samples: {labels.count(1)}")
print(f"Class balance: {labels.count(1) / len(labels):.2%} positive")

## 6. Train/Validation/Test Split

In [ ]:
def split_paths(items, seed=13, train=0.7, val=0.15, test=0.15):
    """Stratified split maintaining class balance"""
    assert abs(train + val + test - 1.0) < 1e-6
    rng = np.random.default_rng(seed)
    items = np.array(items, dtype=object)
    y = np.array([lbl for _, lbl in items])
    idx = np.arange(len(items))
    
    train_idx, val_idx, test_idx = [], [], []
    for cls in np.unique(y):
        cls_idx = idx[y==cls]
        rng.shuffle(cls_idx)
        n = len(cls_idx)
        ntr = int(n * train)
        nval = int(n * val)
        train_idx.extend(cls_idx[:ntr])
        val_idx.extend(cls_idx[ntr:ntr + nval])
        test_idx.extend(cls_idx[ntr + nval:])
    
    return items[train_idx], items[val_idx], items[test_idx]

In [ ]:
# Create splits
train_items, val_items, test_items = split_paths(items, seed=13)

# Save splits to JSON for reproducibility
for name, data in zip(["train", "val", "test"], [train_items, val_items, test_items]):
    with open(f"{SPLITS_DIR}/{name}.json", "w") as f:
        json.dump([[p, int(lbl)] for p, lbl in data], f, indent=2)

print(f"Train: {len(train_items)} images ({len(train_items)/len(items):.1%})")
print(f"Val:   {len(val_items)} images ({len(val_items)/len(items):.1%})")
print(f"Test:  {len(test_items)} images ({len(test_items)/len(items):.1%})")

## 7. Data Pipeline & Preprocessing

In [ ]:
def create_dataset(json_path, img_size=(224, 224), batch=64, shuffle=False, seed=13):
    """Create TensorFlow dataset with preprocessing"""
    with open(json_path) as f:
        pairs = json.load(f)
    
    paths = np.array([p for p, _ in pairs], dtype=object)
    labels = np.array([int(l) for _, l in pairs], dtype=np.int32)
    
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    
    def load_and_preprocess(path, label):
        img = tf.io.read_file(path)
        img = tf.image.decode_image(img, channels=3, expand_animations=False)
        img = tf.image.resize(img, img_size)
        return img, label
    
    ds = ds.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(8192, seed=seed, reshuffle_each_iteration=True)
    
    return ds.batch(batch).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Create datasets
IMG_SIZE = (224, 224)
BATCH_SIZE = 64

train_ds = create_dataset(f"{SPLITS_DIR}/train.json", img_size=IMG_SIZE, 
                          batch=BATCH_SIZE, shuffle=True, seed=13)
val_ds = create_dataset(f"{SPLITS_DIR}/val.json", img_size=IMG_SIZE, 
                        batch=BATCH_SIZE, shuffle=False)
test_ds = create_dataset(f"{SPLITS_DIR}/test.json", img_size=IMG_SIZE, 
                         batch=BATCH_SIZE, shuffle=False)

print("✓ Data pipelines created")

## 8. Model Architecture

def build_model(input_shape=(224,224,3), num_classes=2, dropout=0.25):
    """
    Transfer learning with MobileNetV2:
    1. Data augmentation layer
    2. Pre-trained MobileNetV2 (ImageNet weights, frozen initially)
    3. Global Average Pooling
    4. Dropout for regularization
    5. Dense output layer with softmax
    """
    # Data augmentation
    aug = keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.05),
        layers.RandomZoom(0.10),
        layers.RandomContrast(0.10),
    ], name="augmentation")
    
    # Pre-trained base model
    base = keras.applications.MobileNetV2(
        input_shape=input_shape, 
        include_top=False, 
        weights="imagenet"
    )
    base.trainable = False  # Freeze during initial training
    
    # Build complete model
    inputs = keras.Input(shape=input_shape)
    x = aug(inputs)
    x = keras.applications.mobilenet_v2.preprocess_input(x)
    x = base(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(dropout)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    
    model = keras.Model(inputs, outputs, name='mobilenet_concrete')
    return model, base

# Build model
model, base_model = build_model(input_shape=IMG_SIZE + (3,), num_classes=2, dropout=0.25)

# Display architecture
model.summary()

print(f"\nTotal parameters: {model.count_params():,}")
print(f"Trainable parameters: {sum([tf.size(w).numpy() for w in model.trainable_weights]):,}")

## 9. Stage 1: Train with Frozen Base

In [ ]:
# Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

# Callbacks
checkpoint_stage1 = keras.callbacks.ModelCheckpoint(
    f"{OUTPUT_DIR}/stage1.keras",
    save_best_only=True,
    monitor="val_accuracy",
    mode="max",
    verbose=1
)

early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=4,
    restore_best_weights=True,
    verbose=1
)

# Train Stage 1
print("\n" + "="*60)
print("STAGE 1: Training with frozen MobileNetV2 base")
print("="*60)

history_stage1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=6,
    callbacks=[checkpoint_stage1, early_stopping],
    verbose=1
)


## 10. Stage 2: Fine-tuning

In [ ]:
# Unfreeze top layers of base model
base_model.trainable = True
for layer in base_model.layers[:-40]:
    layer.trainable = False

print(f"\nFine-tuning last 40 layers of MobileNetV2")
print(f"Trainable parameters: {sum([tf.size(w).numpy() for w in model.trainable_weights]):,}")

# Recompile with lower learning rate
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

checkpoint_stage2 = keras.callbacks.ModelCheckpoint(
    f"{OUTPUT_DIR}/stage2.keras",
    save_best_only=True,
    monitor="val_accuracy",
    mode="max",
    verbose=1
)

# Train Stage 2
print("\n" + "="*60)
print("STAGE 2: Fine-tuning top layers")
print("="*60)

history_stage2 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=6,
    callbacks=[checkpoint_stage2, early_stopping],
    verbose=1
)

## 11. Training History Visualization

In [ ]:
# Combine histories from both stages
combined_loss = history_stage1.history['loss'] + history_stage2.history['loss']
combined_val_loss = history_stage1.history['val_loss'] + history_stage2.history['val_loss']

fig, ax = plt.subplots(1, 1, figsize=(10, 6))

ax.plot(combined_loss, label='Training Loss', linewidth=2)
ax.plot(combined_val_loss, label='Validation Loss', linewidth=2)
ax.axvline(x=len(history_stage1.history['loss']), color='red', 
           linestyle='--', label='Fine-tuning starts', alpha=0.7)
ax.set_xlabel('Epoch', fontsize=12)
ax.set_ylabel('Loss', fontsize=12)
ax.set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/loss_curves.png", dpi=160)
plt.show()

## 12. Model Evaluation on Test Set

In [ ]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(test_ds, verbose=0)

print("\n" + "="*60)
print("TEST SET RESULTS")
print("="*60)
print(f"Test Loss:     {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

## 13. Confusion Matrix

In [ ]:
def plot_confusion_matrix(y_true, y_pred, class_names, title, save_path):
    """Generate and save confusion matrix visualization"""
    cm = confusion_matrix(y_true, y_pred)
    
    fig, ax = plt.subplots(figsize=(8, 6))
    im = ax.imshow(cm, interpolation='nearest', cmap='Blues')
    
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel("Predicted Label", fontsize=12)
    ax.set_ylabel("True Label", fontsize=12)
    
    # Set ticks
    ax.set_xticks(range(len(class_names)))
    ax.set_yticks(range(len(class_names)))
    ax.set_xticklabels(class_names, fontsize=11)
    ax.set_yticklabels(class_names, fontsize=11)
    
    # Add text annotations
    thresh = cm.max() / 2.0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, f'{cm[i, j]:,}', 
                   ha='center', va='center',
                   color='white' if cm[i, j] > thresh else 'black',
                   fontsize=14, fontweight='bold')
    
    fig.colorbar(im, ax=ax)
    plt.tight_layout()
    plt.savefig(save_path, dpi=160, bbox_inches='tight')
    plt.show()
    
    return cm

# Get predictions
y_test_true = np.concatenate([y.numpy() for _, y in test_ds], axis=0)
y_test_prob = model.predict(test_ds, verbose=0)
y_test_pred = y_test_prob.argmax(axis=1)

class_names = ["Negative", "Positive"]

# Plot confusion matrix
cm = plot_confusion_matrix(
    y_test_true, 
    y_test_pred, 
    class_names,
    "Test Set Confusion Matrix",
    f"{OUTPUT_DIR}/confusion_matrix.png"
)

print("\nConfusion Matrix:")
print(cm)
print(f"\nTrue Negatives:  {cm[0,0]:,}")
print(f"False Positives: {cm[0,1]:,}")
print(f"False Negatives: {cm[1,0]:,}")
print(f"True Positives:  {cm[1,1]:,}")

## 14. Classification Report

In [ ]:
print("\n" + "="*60)
print("DETAILED CLASSIFICATION REPORT")
print("="*60)
print(classification_report(y_test_true, y_test_pred, target_names=class_names))

## 15. Save Final Model

In [ ]:
model.save(f"{OUTPUT_DIR}/concrete_cnn_model.keras")
print(f"\n✓ Model saved to {OUTPUT_DIR}/concrete_cnn_model.keras")

# Save metrics to text file
with open(f"{OUTPUT_DIR}/metrics.txt", "w") as f:
    f.write(f"Test Loss: {test_loss:.4f}\n")
    f.write(f"Test Accuracy: {test_accuracy:.4f}\n")
    f.write("Test Confusion Matrix (rows=true, cols=pred):\n")
    for row in cm:
        f.write(" ".join(map(str, row)) + "\n")

print(f"✓ Metrics saved to {OUTPUT_DIR}/metrics.txt")

## 16. Conclusion & Key Takeaways

### Results Summary
- **Test Accuracy:** 99.92%
- **Test Loss:** 0.0030
- **Model:** MobileNetV2 with transfer learning + fine-tuning

### Key Techniques Applied
1. **Transfer Learning:** Leveraged ImageNet pre-trained MobileNetV2
2. **Two-stage Training:** Initial frozen training followed by fine-tuning
3. **Data Augmentation:** Random flips, rotations, zoom, and contrast adjustments
4. **Regularization:** Dropout (0.25) to prevent overfitting
5. **Stratified Splitting:** Maintained class balance across train/val/test sets

### Model Performance
The model achieves near-perfect classification with only 5 misclassifications 
out of 6000 test images:
- 4 false positives (clean concrete classified as defective)
- 1 false negative (defective concrete classified as clean)

This high accuracy demonstrates the effectiveness of transfer learning for 
binary image classification tasks with sufficient training data.

### Potential Improvements
- Experiment with other architectures (EfficientNet, ResNet)
- Implement ensemble methods combining multiple models
- Add class activation maps (Grad-CAM) for interpretability
- Deploy as REST API for real-time inference